In [1]:
import arcpy

sde_conn = r"C:\opt\work\igac\data\egdb_conn\colsmart_dev\egdb_colsmart_dev_colsmart_test5_owner.sde"
arcpy.env.workspace = sde_conn
print(sde_conn)
#desc = arcpy.Describe(sde_conn)
#print(desc)

# sde_conn+"\\"+
rel = "edgeograficos.colsmart_test5_owner.CR_UND_CON_CR_FUE_ESP"
print(rel)
desc = arcpy.Describe(rel)

print(desc)

# Print relationship class properties
#
print ("Backward Path Label: {}".format(desc.backwardPathLabel))
print ("Cardinality: {}".format(desc.cardinality))
print ("Class key: {}".format(desc.classKey))
print ("Destination Class Names: {}".format(desc.destinationClassNames))
print ("Forward Path Label: {}".format(desc.forwardPathLabel)) 
print ("Is Attributed: {}".format(desc.isAttributed))
print ("Is Composite: {}".format(desc.isComposite)) 
print ("Is Reflexive: {}".format(desc.isReflexive))
print ("Key Type: {}".format(desc.keyType))
print ("Notification Direction: {}".format(desc.notification))
print ("Origin Class Names: {}".format(desc.originClassNames))

C:\opt\work\igac\data\egdb_conn\colsmart_dev\egdb_colsmart_dev_colsmart_test5_owner.sde
edgeograficos.colsmart_test5_owner.CR_UND_CON_CR_FUE_ESP
<geoprocessing describe data object object at 0x000001F93539EBD0>
Backward Path Label: CR_UnidadConstruccion
Cardinality: OneToMany
Class key: Undefined
Destination Class Names: ['edgeograficos.colsmart_test5_owner.CR_FuenteEspacial']
Forward Path Label: CR_FuenteEspacial
Is Attributed: False
Is Composite: False
Is Reflexive: False
Key Type: Single
Notification Direction: None
Origin Class Names: ['edgeograficos.colsmart_test5_owner.CR_UnidadConstruccion']


In [2]:
print(f"desc.originClassKeys:  {desc.originClassKeys}")
print(f"desc.destinationClassKeys:  {desc.destinationClassKeys}")

desc.originClassKeys:  [('globalid', 'OriginPrimary', ''), ('UnidadConstruccion_GUID', 'OriginForeign', '')]
desc.destinationClassKeys:  []


In [5]:
type(desc.originClassKeys)

list

In [3]:
for i in desc.originClassKeys:
    print (i)
    print(type(i))
    print(i[1])
    print(i[0])

('globalid', 'OriginPrimary', '')
<class 'tuple'>
OriginPrimary
globalid
('UnidadConstruccion_GUID', 'OriginForeign', '')
<class 'tuple'>
OriginForeign
UnidadConstruccion_GUID


In [4]:
elements = arcpy.ListTables() + arcpy.ListFeatureClasses() 
print(elements)

['edgeograficos.colsmart_test5_owner.CR_DatosPHCondominio', 'edgeograficos.colsmart_test5_owner.CR_FuenteEspacial', 'edgeograficos.colsmart_test5_owner.ExtDireccion', 'edgeograficos.colsmart_test5_owner.ILC_CaracteristicasUnidadConstruccion', 'edgeograficos.colsmart_test5_owner.ILC_DatosAdicionalesLevantamientoCatastral', 'edgeograficos.colsmart_test5_owner.ILC_Derecho', 'edgeograficos.colsmart_test5_owner.ILC_EstructuraAvaluo', 'edgeograficos.colsmart_test5_owner.ILC_FuenteAdministrativa', 'edgeograficos.colsmart_test5_owner.ILC_Interesado', 'edgeograficos.colsmart_test5_owner.ILC_Predio_Informalidad', 'edgeograficos.colsmart_test5_owner.ILC_TramitesDerechosTerritoriales', 'edgeograficos.colsmart_test5_owner.CR_Terreno', 'edgeograficos.colsmart_test5_owner.CR_UnidadConstruccion', 'edgeograficos.colsmart_test5_owner.ILC_Marcas', 'edgeograficos.colsmart_test5_owner.ILC_Predio']


In [5]:
# --- Validación de dominios
elements = arcpy.ListTables() + arcpy.ListFeatureClasses() 
print(elements)
for fc in elements:
    print(fc)
    fields = arcpy.ListFields(fc)
    for f in fields:
        if f.domain:
            print(f)
            valores_validos = [val[0] for val in arcpy.da.ListDomain(gdb_path, f.domain)["codedValues"]]
            with arcpy.da.SearchCursor(fc, [f.name]) as cursor:
                for row in cursor:
                    if row[0] not in valores_validos:
                        registrar(f"Valor fuera de dominio: {row[0]}", "Dominio", fc, f.name, str(row[0]))

edgeograficos.colsmart_test5_owner.CR_DatosPHCondominio
<geoprocessing describe field object object at 0x000001F93539E590>


<class 'AttributeError'>: module 'arcpy.da' has no attribute 'ListDomain'

In [6]:
import arcpy

def registrar(mensaje, tipo, tabla, campo, valor, objectid):
    print(f"[{tipo}] {mensaje} → Tabla: {tabla}, Campo: {campo}, Valor: {valor}, objectid: {objectid}")

sde_conn = r"C:\opt\work\igac\data\egdb_conn\colsmart_dev\egdb_colsmart_dev_colsmart_test5_owner.sde"
arcpy.env.workspace = sde_conn

# Obtener todos los dominios en la geodatabase
dominios = arcpy.da.ListDomains()
dominios_dict = {dom.name: dom for dom in dominios if dom.domainType == 'CodedValue'}
print(len(dominios_dict))

# Obtener todas las tablas y feature classes
elementos =  arcpy.ListTables() + arcpy.ListFeatureClasses() 
print(len(elementos))

for fc in elementos:
    print("***************************")
    print(f"Validando: {fc}")
    fields = arcpy.ListFields(fc)
    for f in fields:
        if f.domain and f.domain in dominios_dict:
            #print(f)
            dominio = dominios_dict[f.domain]
            print(dominio.name)
            valores_validos = list(dominio.codedValues.keys())
            print(valores_validos)

            with arcpy.da.SearchCursor(fc, [f.name, "objectid"]) as cursor:
                for row in cursor:
                    #print(row)
                    if row[0] is not None and row[0] not in valores_validos:
                        registrar("Valor fuera de dominio", "Dominio", fc, f.name, row[0], row[1])
                    break
            break


76
15
***************************
Validando: edgeograficos.colsmart_test5_owner.CR_DatosPHCondominio
***************************
Validando: edgeograficos.colsmart_test5_owner.CR_FuenteEspacial
<geoprocessing describe field object object at 0x00000268D6BCB7F0>
COL_SiNo
['No', 'Si']
***************************
Validando: edgeograficos.colsmart_test5_owner.ExtDireccion
<geoprocessing describe field object object at 0x00000268D4070A50>
ILC_ClaseViaPrincipalTipo
['Autopista', 'Avenida', 'Avenida_Calle', 'Avenida_Carrera', 'Calle', 'Carrera', 'Circular', 'Circunvalar', 'Diagonal', 'Transversal']
***************************
Validando: edgeograficos.colsmart_test5_owner.ILC_CaracteristicasUnidadConstruccion
<geoprocessing describe field object object at 0x00000268D6BCB950>
CUC_EstadoConservacionTipologiaTipo
['Bueno', 'Deficiente', 'Intermedio', 'Malo', 'Muy_Bueno', 'Muy_Malo', 'Optimo', 'Regular', 'Ruinoso']
***************************
Validando: edgeograficos.colsmart_test5_owner.ILC_DatosAd